# Machine Learning Workflow
Complete the steps below to complete the machine learning workflow for this classifier.

In [1]:
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [47]:
import re
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix

In [48]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def load_data():
    df = pd.read_csv('corporate_messaging.csv', encoding='latin-1')
    df = df[(df["category:confidence"] == 1) & (df['category'] != 'Exclude')]
    X = df.text.values
    y = df.category.values
    return X, y

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### Step 1: Load data and perform a train test split

In [49]:
# load data
X, y = load_data()

# perform train test split
X_train, X_test, y_train ,  y_test = train_test_split(X,y)

In [50]:
print(X_train.shape, y_train.shape , X_test.shape, y_test.shape)

(1802,) (1802,) (601,) (601,)


### Step 2: Train classifier
* Fit and transform the training data with `CountVectorizer`. Hint: You can include your tokenize function in the `tokenizer` keyword argument!
* Fit and transform these word counts with `TfidfTransformer`.
* Fit a classifier to these tfidf values.

In [51]:
# Instantiate transformers and classifier
vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer()
clf =  RandomForestClassifier()

# Fit and/or transform each to the data

X_train_count = vect.fit_transform(X_train)
X_train_tfidf = tfidf.fit_transform(X_train_count)
clf.fit(X_train_tfidf, y_train)




RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

### convert sparse matrix to numpy array to view


In [52]:
X_train_count.toarray()[0].shape, X_train_count.toarray()[0]

((5549,), array([0, 2, 0, ..., 0, 0, 0], dtype=int64))

In [53]:
vect.vocabulary_

{'rbc': 4099,
 'lure': 3139,
 'wall': 5246,
 'street': 4730,
 'debt': 1538,
 'trader': 4989,
 'with': 5356,
 'volume': 5203,
 'in': 2690,
 'u.s.': 5060,
 'doubling': 1698,
 ':': 330,
 'urlplaceholder': 5124,
 '#': 1,
 'citigroup': 1162,
 'brk': 916,
 'valentin': 5146,
 'marinov': 3200,
 ',': 42,
 'citi': 1150,
 'chief': 1117,
 'g10': 2270,
 'fx': 2267,
 'strategist': 4728,
 'will': 5340,
 'be': 734,
 'on': 3626,
 'business': 946,
 'news': 3514,
 'network': 3502,
 'at': 636,
 '11:45': 98,
 'gmt': 2358,
 'to': 4939,
 'discus': 1653,
 'the': 4885,
 'trading': 4990,
 'range': 4091,
 '@': 334,
 'bnn': 845,
 'according': 376,
 'a': 340,
 'pfizer-sponsored': 3812,
 'survey': 4791,
 '52': 264,
 '%': 3,
 'of': 3608,
 'woman': 5364,
 'metastatic': 3286,
 'breast': 899,
 'cancer': 990,
 '(': 36,
 'mbc': 3229,
 ')': 37,
 'believe': 758,
 'receives': 4128,
 'too': 4953,
 'little': 3085,
 'public': 4044,
 'attention': 647,
 'kelimccain': 2917,
 'thank': 4880,
 'you': 5456,
 'for': 2176,
 'volunteeri

### Step 3: Predict on test data
* Transform (no fitting) the test data with the same CountVectorizer and TfidfTransformer
* Predict labels on these tfidf values.

In [54]:
# Transform test data
X_test_count = vect.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_count)

# Predict test labels
y_pred = clf.predict(X_test_tfidf)

### Step 4: Display results
Display a confusion matrix and accuracy score based on the model's predictions.

In [55]:
labels = np.unique(y_test)
confusion_mat = confusion_matrix(y_test, y_pred,labels=labels)
accuracy = (y_test==y_pred).mean()

print("Labels:", labels)
print("Confusion Matrix:\n", confusion_mat)
print("Accuracy:", accuracy)

Labels: ['Action' 'Dialogue' 'Information']
Confusion Matrix:
 [[ 84   0  32]
 [  2  27   4]
 [ 13   1 438]]
Accuracy: 0.913477537438


# Final Step: Refactor
Organize these steps into the following functions.

In [57]:
def display_results(y_test, y_pred):
    # insert step 4 here 
    labels = np.unique(y_test)
    confusion_mat = confusion_matrix(y_test, y_pred,labels=labels)
    accuracy = (y_test==y_pred).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)



def main():
    # load data
    X, y = load_data()

    # perform train test split
    X_train, X_test, y_train ,  y_test = train_test_split(X,y)
    
    # Instantiate transformers and classifier
    vect = CountVectorizer(tokenizer=tokenize)
    tfidf = TfidfTransformer()
    clf =  RandomForestClassifier()

    # Fit and/or transform each to the data

    X_train_count = vect.fit_transform(X_train)
    X_train_tfidf = tfidf.fit_transform(X_train_count)
    clf.fit(X_train_tfidf, y_train)
    
    # Transform test data
    X_test_count = vect.transform(X_test)
    X_test_tfidf = tfidf.transform(X_test_count)

    # Predict test labels
    y_pred = clf.predict(X_test_tfidf)
    
    display_results(y_test, y_pred)


In [58]:
# run program
main()

Labels: ['Action' 'Dialogue' 'Information']
Confusion Matrix:
 [[ 83   0  28]
 [  3  22   9]
 [  6   3 447]]
Accuracy: 0.91846921797
